# SETUP

In [1]:
from requests import get
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from IPython.display import HTML
import pyodbc
import time
from sklearn.linear_model import LogisticRegression

from pandas.io.json import json_normalize

In [2]:
from mycbr_py_api import MyCBRRestApi as mycbr

# Methods

In [3]:
casebase=list()
initCBCopy=casebase
currentCBCopy=casebase


#Retrieves data for the experiment
data00=()
def readData():
    global data00
    cnxn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=RisikoIndeksTjenesten;Trusted_Connection=yes;')
    cursor = cnxn.cursor()
    data00=pd.read_sql("SELECT  * "+"FROM [RisikoIndeksTjenesten].[dbo].[BayesianDynamicChecklistLocalDb]"+
                      "left join (select NewID() as new, InspectionId as inspID from [RisikoIndeksTjenesten].[dbo].[BayesianDynamicChecklistLocalDb] group by InspectionId) as a on a.inspId=InspectionId"+
                      " where InspectionDateId<20190601 ORDER BY new",cnxn)
    #Query selects items in random order, grouped by the inspectionIDs. 
    #We are doing crossvalidation with random split based on the indices of the retrieved items.

#create dataframe for evaluating existing checklists
def fillCaseBase(splitindex):
    global casebase
    global data00

    data0=data00
    datatestout=data0.loc[round((len(data0)/8)*splitindex):round((len(data0)/8)*(splitindex+1))]

    casebase=datatestout
    

#Restores the data to original form
def restoreOriginalCaseBase(splitindex):
    fillCaseBase(splitindex)
    #return res;


# EXPERIMENT, STATISTICS FOR GENERATED CHECKLISTS

In [6]:
#Control point is synonymous with checklist item
initCBCopy=list();
currentCBCopy=initCBCopy
readData()
timetotal=0
traintime=0
precctot=0
accctot=0
preccgttot=0
recctot=0

fh2=open("Log_" + "ORGChecklistsTest"+ '.txt', 'w+')
for ik in range(0,8):
    
    start_time = time.time()
    if len(initCBCopy)==0:
        fillCaseBase(ik)
        initCBCopy=casebase
        currentCBCopy=initCBCopy
    else:
        restoreOriginalCaseBase(ik)
        initCBCopy=casebase
        currentCBCopy=initCBCopy
    traintime=traintime+(time.time()-start_time)
    print("Train time: "+str(traintime/(ik+1)))
    noncompliance=0 #true positive(gt)
    controlpoints=0 #predicted positive(gt)
    noncompliancengt=0 #true positive (ngt)
    controlpointsngt=0#predicted positive (ngt)
    truepositiveval=0
    truepositive=0
    falsepositivegt=0
    falsepositivengt=0
    precisionval=0
    recallval=0
    accuracyval=0
    precision=0
    precisiongt=0
    lengthprecgt=0
    recall=0
    accuracy=0
    lengthprec=0.00001
    lengthvalprec=0
    lengthrec=0.00001
    lengthvalrec=0
    lengthacc=0.00001
    lengthvalacc=0
    truenegative=0
    falsenegative=0
    similarity=0
    counter=0
    Kcp=15
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', -1)
    noncompliantInspection=0
    numberofOverlappingControlpoints=0
    inspections=casebase.drop_duplicates(subset = ["InspectionId"])
    cases=casebase
    cases["NonCompliance"]=cases["NonCompliance"].astype(int)
    casestest=len(cases.drop_duplicates(subset = ["ControlPointText"]))
    cpCases=list()
    negatives=list()
    cltotallen=0
    inspectionstot=0
    precchecklists=0
    recchecklists=0
    accchecklists=0
    precchecklistsval=0
    recchecklistsval=0
    accchecklistsval=0
    print(casestest)
    for ind, case in inspections.iterrows():
       
        inspectionstot+=1
        uniquechecklistlengthval=0
        cpCases=list()
        counter+=1
        if counter>500:
            counter=0


            
        r=0#Stop variable to ensure at least 10 cases are selected for each inspection (case)
        k=500#Finding the Top K cases
        
        #Existing checklists are retrieved via their inspectionId
        existingChecklist=cases[cases['InspectionId']==case['InspectionId']]#Retrieve the existing checklist
        clck=cases[cases["IndustrySubgroupCode"].astype(float)==float(case["IndustrySubgroupCode"])]
        clck=clck[clck["MunicipalityNumber"].astype(int)==int(case["MunicipalityNumber"])]
        clck["NonCompliance"]=clck["NonCompliance"].astype(float)
        negatives=clck
        if len(clck)>0:
            caseAvgNoncompliancePrControlpoint=clck.groupby(["ControlPointText"],as_index=False).mean()

        while r==0:
            checklist=existingChecklist#Just using the existing checklist

            uniqueChecklist=checklist.drop_duplicates(subset = ["ControlPointText"])#Find all unique items by removing duplicates

            if len(uniqueChecklist)>(0):#Just checking to make sure that the retrieved checklist is not empty
                truepositive=0
                truepositiveval=0
                r=1#Stops the while loop since 10 unique items are found
 
                precpercl=0
                lengthorgcl=0
                for ind2, generatedChecklistControlpoint in uniqueChecklist.iterrows():#Find overlap between the existing and new generated checklist
                    cltotallen+=1
                    negatives=negatives[negatives["ControlPointText"]!=generatedChecklistControlpoint["ControlPointText"]]#This is wrong. Negatives need to be outside the whole checklist
                    excp=existingChecklist[existingChecklist["ControlPointText"]==generatedChecklistControlpoint["ControlPointText"]]
                    if len(excp)>0:
                        numberofOverlappingControlpoints+=1 
                        precpercl+=((excp["NonCompliance"].sum())/len(excp["NonCompliance"]))
                        lengthorgcl+=1
                        controlpoints+=1#matchlen
                        

                    caseAvgNoncompliancePrControlpoint2=caseAvgNoncompliancePrControlpoint[caseAvgNoncompliancePrControlpoint["ControlPointText"]==generatedChecklistControlpoint["ControlPointText"]]
                    
                    summ2=caseAvgNoncompliancePrControlpoint2["NonCompliance"].sum() 
                    if len(caseAvgNoncompliancePrControlpoint2["NonCompliance"])>0:
                        noncompliance+=(summ2/len(caseAvgNoncompliancePrControlpoint2["NonCompliance"]))
                        truepositiveval+=(summ2/len(caseAvgNoncompliancePrControlpoint2["NonCompliance"]))
                    
                    matchlen=len(caseAvgNoncompliancePrControlpoint2["NonCompliance"])
                    if matchlen>1:
                        print(caseAvgNoncompliancePrControlpoint2["NonCompliance"])
                    controlpointsngt+=1
                    falsepositivegt+=(matchlen-summ2)
                    if matchlen>0:
                        precisionval+=(summ2/matchlen)
                        lengthvalprec+=1
                        uniquechecklistlengthval+=1
                    summ3=0
                    if matchlen==0:#if validation record of checklist item does not exist
                        cpCases=cases[cases["IndustrySubgroupCode"].astype(float)==float(generatedChecklistControlpoint["IndustrySubgroupCode"])]
                        cpCases=cpCases[cpCases["MunicipalityNumber"].astype(int)==int(generatedChecklistControlpoint["MunicipalityNumber"])]
                        cpCases=cpCases[cpCases["ControlPointText"]==generatedChecklistControlpoint["ControlPointText"]]
                        cpCases["NonCompliance"]=cpCases["NonCompliance"].astype(float)
                        summ3=cpCases["NonCompliance"].sum()
                        
                        
                        falsepositivengt+=(len(cpCases["NonCompliance"])-summ3)
                        if len(cpCases["NonCompliance"])>0:
                            noncompliancengt+=(summ3/len(cpCases["NonCompliance"]))
                            truepositive+=(summ3/len(cpCases["NonCompliance"]))
                            precision+=(summ3/len(cpCases["NonCompliance"]))
                            lengthprec+=1
                if lengthorgcl>0:
                    precisiongt+=(precpercl/lengthorgcl)
                    lengthprecgt+=1      

                #Calculate statistics on checklists level
                uniquenegatives=len(negatives.drop_duplicates(subset = ["ControlPointText"]))
                negativecpy=negatives.copy()
                negativecpy["NonCompliance"]=negativecpy["NonCompliance"].astype(float)
                groupedby=negativecpy.groupby(["ControlPointText"],as_index=False).count()
                groupedbys=negativecpy.groupby(["ControlPointText"],as_index=False).sum()
                groupedby["NonCompliance"]=groupedbys["NonCompliance"]/groupedby["NonCompliance"]
                uniquecalcnegatives=groupedby.drop_duplicates(subset = ["ControlPointText"])
                noncompliancenegatives=uniquecalcnegatives["NonCompliance"].sum()

                falsenegativeelement=0
                if uniquenegatives>0:
                    falsenegativeelement=noncompliancenegatives/uniquenegatives #To avoid selection bias effects
                truenegativeelement=1-falsenegativeelement
                
                truepositivesprchecklistval=(truepositiveval)
                falsepositivesprchecklistval=(uniquechecklistlengthval-truepositivesprchecklistval)
                
                truepositivesprchecklist=(truepositive+truepositiveval)
                falsepositivesprchecklist=(len(uniqueChecklist)-(truepositive+truepositiveval))
                truenegativesprchecklist=truenegativeelement*uniquenegatives
                falsenegativesprchecklist=falsenegativeelement*uniquenegatives
                
                precprchecklist=truepositivesprchecklist/len(uniqueChecklist)
                precchecklists+=precprchecklist
                #print("prec: "+str(precprchecklist))
                precprchecklistval=0
                if uniquechecklistlengthval>0:
                    precprchecklistval=truepositivesprchecklistval/uniquechecklistlengthval
                precchecklistsval+=precprchecklistval
                #print("prec(val): "+str(precprchecklistval))
                recprchecklist=0
                if (truepositivesprchecklist+falsenegativesprchecklist)>0:
                    recprchecklist=truepositivesprchecklist/(truepositivesprchecklist+falsenegativesprchecklist)
                recchecklists+=recprchecklist
               # print("rec: "+str(recprchecklist))
                recprchecklistval=0
                if (truepositivesprchecklistval+falsenegativesprchecklist)>0:
                    recprchecklistval=truepositivesprchecklistval/(truepositivesprchecklistval+falsenegativesprchecklist)
                recchecklistsval+=recprchecklistval
                #print("rec(val): "+str(recprchecklistval))
                accprchecklist=0
                if (len(uniqueChecklist)+uniquenegatives)>0:
                    accprchecklist=(truepositivesprchecklist+truenegativesprchecklist)/(len(uniqueChecklist)+uniquenegatives)
                accchecklists+=accprchecklist
                #print("acc: "+str(accprchecklist))
                accprchecklistval=0
                if (uniquechecklistlengthval+uniquenegatives)>0:
                    accprchecklistval=(truepositivesprchecklistval+truenegativesprchecklist)/(uniquechecklistlengthval+uniquenegatives)
                accchecklistsval+=accprchecklistval
                #print("acc(val): "+str(accprchecklistval))
                              
                #|true positives for each checklist|=sum(true positive elements in checklist)
                #|false positives for each checklist|=sum(false positive elements in checklist)
                #|true negatives for each checklist|=|true negative element|*|unique items(2)|
                #|false negatives for each checklist|=|false negative element|*|unique items(2)|
                #prec pr checklist
                #rec pr checklist
                #acc pr checklist
                        
            k+=500 #Increment k while there are less then 10 unique items
    if inspectionstot>0:
        precctot+=precchecklists/inspectionstot
        accctot+=accchecklists/inspectionstot
        preccgttot+=precisiongt/lengthprecgt
        recctot+=recchecklists/inspectionstot
       
    timetotal=timetotal+(time.time()-start_time)
    print("Current avg time:"+ str(timetotal/(ik+1)))
    fh2.write("Current avg time:"+ str(timetotal/(ik+1)))



#The average accuracy, precision and recall scores from the cross-validation. These numbers are used in the paper. 
print("\nAverage Accuracy: "+str(accctot/8))
print("\nAverage Prec: "+str(precctot/8))
print("\nAverage Rec: "+str(recctot/8))
print("\nAverage Precgt: "+str(preccgttot/8))
fh2.write("\nAverage Accuracy: "+str(accctot/8))
fh2.write("\nAverage Prec: "+str(precctot/8))
fh2.write("\nAverage Rec: "+str(recctot/8))
fh2.write("\nAverage Precgt: "+str(preccgttot/8))
fh2.close()

Train time: 0.0


<ipython-input-6-d90cac3f0bbc>:56: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-6-d90cac3f0bbc>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases["NonCompliance"]=cases["NonCompliance"].astype(int)


974
Current avg time:297.2115206718445
Train time: 0.0005015134811401367
971


<ipython-input-6-d90cac3f0bbc>:56: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-6-d90cac3f0bbc>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases["NonCompliance"]=cases["NonCompliance"].astype(int)


Current avg time:310.0922636985779
Train time: 0.00033434232076009113
980


...

Current avg time:309.0442586342494
Train time: 0.00014328956604003906
955


<ipython-input-6-d90cac3f0bbc>:56: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-6-d90cac3f0bbc>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases["NonCompliance"]=cases["NonCompliance"].astype(int)


Current avg time:308.64940697806225
Train time: 0.00037544965744018555
971


<ipython-input-6-d90cac3f0bbc>:56: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-6-d90cac3f0bbc>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases["NonCompliance"]=cases["NonCompliance"].astype(int)


Current avg time:306.0137441456318

Average Accuracy: 0.3663422107543029

Average Prec: 0.17638049179838536

Average Rec: 0.5916204549347978

Average Precgt: 0.1768967810278069
